In [ ]:
import time
import redis
import xmltodict
from mlrepricer import parser, helper, setup, minmax, listener, redisdb
import mws
import threading
from ruamel.yaml import YAML

yaml = YAML()
mwsid = helper.mwscred['account_id']
decimal = setup.decimal

r = redis.StrictRedis(**helper.rediscred, decode_responses=True)
mapping = minmax.load_csv()

In [ ]:
redisdb.server_start()

In [ ]:
m = r.srandmember('updated_asins')
m

In [ ]:
result = []
for asin in r.scan_iter():
    if asin != 'updated_asins':
        for message in r.zscan_iter(asin):
            result.append(len(message[0]))
    else:
        for newasin in r.sscan_iter('updated_asins'):
            print(newasin)

In [ ]:
min(result)

In [ ]:
r.keys('*')

In [ ]:
listener.Listener().start()

In [ ]:
for va in r.scan('updated_asins'):
    print(va)

In [ ]:
for asin in r.smembers('updated_asins'):
    print(asin)

In [ ]:
for key in r.scan_iter():
       print(key)

In [ ]:
def get_latest_message(asin):
    """
    Leverage the structure of redis zsets, sorted sets with score.

    Return the parsed and flattend message.
    """
    m = r.zrevrangebyscore(asin, 'inf', '-inf', start=0, num=1,
                           withscores=True)
    return parser.main(xmltodict.parse(m[0][0]))


def get_buyboxwinner(parsedxml):
    """
    Return one or multiple winner for prime and not prime offers.

    If we are not in the buybox.
    Format: ([prime_offerobject, ...], [nonprime_offerobject, ...])
    """
    prime_winner, nonprime_winner = [], []
    for x in parsedxml:
        if (x['sellerid'] == mwsid) & (x['isbuyboxwinner']):
            return [], []  # if we are winning, we are happy
        elif x['isbuyboxwinner'] & x['isprime']:
            prime_winner.append(x)
        elif x['isbuyboxwinner'] & ~x['isprime']:
            nonprime_winner.append(x)
    return prime_winner, nonprime_winner


def get_sku(asin):
    """We assume there is you have max one offer per type."""
    m = mapping.asin == asin
    prime_offer = list(mapping[m & (mapping.isprime)].seller_sku.values)
    nonprime_offer = list(mapping[m & ~(mapping.isprime)].seller_sku.values)
    return (prime_offer, nonprime_offer)


def matchprice(sku, winner):
    """Primitive repricing rule."""
    for x, y in zip(sku, winner):
        if x and y:  # When the buybox is the same type as our listing
            sellersku = x[0]
            p = [c['price'] for c in y]
            buyboxprice = round(sum(p)/float(len(p)), 2)
            return sellersku, buyboxprice


def boundaries(sku, buyboxprice):
    df = minmax.load_csv()


def create_feed(products_to_update):
    """Process a tsv file for the mws feeds api."""
    feed_header = 'sku\tprice\n'
    feed_row_list = []

    for product in products_to_update:
        if product is not None:
            # the feedrow format: sku\tprice
            feed_row = f"{product[0]}\t{str(product[1]).replace('.', decimal)}"
            feed_row_list.append(feed_row)

    feed_data = feed_header + '\n'.join(feed_row_list)
    return feed_data.encode('utf8')


def main():
    """Pop the list of changed asins and take action on them."""
    starttime = time.time()
    while True:
        products_to_update = []
        for asin in r.smembers('updated_asins'):
            print(asin)
            # r.srem(asin)  # that should be ok compared to other options
            m = get_latest_message(asin)
            time_changed = m['time_changed']
            winner = get_buyboxwinner(m)
            skutuple = get_sku(asin)
            sku, buyboxprice = matchprice(skutuple, winner)
            products_to_update.append(sku, buyboxprice)
            # we store the action in redis
            # need to find better structure for easy access TODO
            r.sadd(
                'actions', yaml.dump([time_changed, asin, sku, buyboxprice]))

        feed_data = create_feed(products_to_update)
        feeds_api = mws.Feeds(**helper.mwscred)
        return feeds_api.submit_feed(
            feed_data, '_POST_FLAT_FILE_INVLOADER_DATA_')
        # We send maximum every 30 seconds a new feed to the mws api.
        time.sleep(30.0 - ((time.time() - starttime) % 30.0))

In [ ]:
main()